![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/openvino/HuggingFace_OpenVINO_in_Spark_NLP_XLM-RoBERTa.ipynb)

# Import OpenVINO XLM-RoBERTa models from HuggingFace 🤗 into Spark NLP 🚀

This notebook provides a detailed walkthrough on optimizing and exporting XlmRoBerta models from HuggingFace for use in Spark NLP, leveraging the various tools provided in the [Intel OpenVINO toolkit](https://www.intel.com/content/www/us/en/developer/tools/openvino-toolkit/overview.html) ecosystem.

Let's keep in mind a few things before we start 😊

- OpenVINO support was introduced in  `Spark NLP 5.4.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for XLM-RoBERTa from HuggingFace and they have to be in `Fill Mask` category. Meaning, you cannot use XLM-RoBERTa models trained/fine-tuned on a specific task such as token/sequence classification.

## 1. Export and Save HuggingFace model

- Let's install `transformers` and `openvino` packages with other dependencies. You don't need `openvino` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.41.2`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install -q --upgrade transformers==4.41.2
!pip install -q --upgrade openvino==2024.1
!pip install -q --upgrade optimum-intel
!pip install -q --upgrade sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.7/215.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.7/38.7 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.4/418.4 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

[Optimum Intel](https://github.com/huggingface/optimum-intel?tab=readme-ov-file#openvino) is the interface between the Transformers library and the various model optimization and acceleration tools provided by Intel. HuggingFace models loaded with optimum-intel are automatically optimized for OpenVINO, while being compatible with the Transformers API.
- To load a HuggingFace model directly for inference/export, just replace the `AutoModelForXxx` class with the corresponding `OVModelForXxx` class. We can use this to import and export OpenVINO models with `from_pretrained` and `save_pretrained`.
- By setting `export=True`, the source model is converted to OpenVINO IR format on the fly.
- We'll use [xlm-roberta-base](https://huggingface.co/xlm-roberta-base) model from HuggingFace as an example and load it directly as an `OVModelForFeatureExtraction`, representing an OpenVINO model. By setting `export=True`, the available source model is converted to the OpenVINO IR format on the fly.
- In addition to the XLM-RoBERTa model, we also need to save the `XLMRobertaTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from optimum.intel import OVModelForFeatureExtraction
from transformers import XLMRobertaTokenizer

MODEL_NAME = "xlm-roberta-base"
EXPORT_PATH = f"ov_models/{MODEL_NAME}"

ov_model = OVModelForFeatureExtraction.from_pretrained(MODEL_NAME, export=True)
tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL_NAME)

# Save the OpenVINO model
ov_model.save_pretrained(EXPORT_PATH)
tokenizer.save_pretrained(EXPORT_PATH)

# Create directory for assets and move the tokenizer files.
# A separate folder is needed for Spark NLP.
!mkdir {EXPORT_PATH}/assets
!mv {EXPORT_PATH}/sentencepiece.bpe.model {EXPORT_PATH}/assets/

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

Framework not specified. Using pt to export the model.


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Using framework PyTorch: 2.3.0+cu121
Overriding 1 configuration item(s)
	- use_cache -> False
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4481: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:2246: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert padding_idx < weight.size(0), "Padding_idx must be within num_embeddings"
Compiling the model to CPU ...


Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {EXPORT_PATH}

total 1084220
drwxr-xr-x 2 root root       4096 Jun  5 17:40 assets
-rw-r--r-- 1 root root        679 Jun  5 17:40 config.json
-rw-r--r-- 1 root root 1109816508 Jun  5 17:40 openvino_model.bin
-rw-r--r-- 1 root root     400993 Jun  5 17:40 openvino_model.xml
-rw-r--r-- 1 root root        280 Jun  5 17:40 special_tokens_map.json
-rw-r--r-- 1 root root       1172 Jun  5 17:40 tokenizer_config.json


In [4]:
!ls -l {EXPORT_PATH}/assets

total 4952
-rw-r--r-- 1 root root 5069051 Jun  5 17:40 sentencepiece.bpe.model


## 2. Import and Save XLM-RoBERTa in Spark NLP

- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [ ]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Let's start Spark with Spark NLP included via our simple `start()` function

In [ ]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

- Let's use `loadSavedModel` functon in `XlmRoBertaEmbeddings` which allows us to load the OpenVINO model
- Most params will be set automatically. They can also be set later after loading the model in `XlmRoBertaEmbeddings` during runtime, so don't worry about setting them now
- `loadSavedModel` accepts two params, first is the path to the exported model. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- `setStorageRef` is very important. When you are training a task like NER or any Text Classification, we use this reference to bound the trained model to this specific embeddings so you won't load a different embeddings by mistake and see terrible results 😊
- It's up to you what you put in `setStorageRef` but it cannot be changed later on. We usually use the name of the model to be clear, but you can get creative if you want!
- The `dimension` param is is purely cosmetic and won't change anything. It's mostly for you to know later via `.getDimension` what is the dimension of your model. So set this accordingly.
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.


In [7]:
from sparknlp.annotator import *

# All these params should be identical to the original model
xlm_roberta = XlmRoBertaEmbeddings.loadSavedModel(f"{EXPORT_PATH}", spark)\
    .setInputCols(["document",'token'])\
    .setOutputCol("xlm_roberta")\
    .setCaseSensitive(True)\
    .setDimension(768)\
    .setStorageRef('xlm_roberta_base')

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [8]:
xlm_roberta.write().overwrite().save(f"{MODEL_NAME}_spark_nlp")

Let's clean up stuff we don't need anymore

In [9]:
!rm -rf {EXPORT_PATH}

Awesome  😎 !

This is your OpenVINO XLM-RoBERTa model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [10]:
! ls -l {MODEL_NAME}_spark_nlp

total 1089324
drwxr-xr-x 2 root root       4096 Jun  5 17:52 metadata
-rw-r--r-- 1 root root 1110387189 Jun  5 17:53 xlmroberta_openvino
-rw-r--r-- 1 root root    5069051 Jun  5 17:53 xlmroberta_spp


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny XLM-RoBERTa model 😊

In [11]:
import sparknlp

from sparknlp.base import *
from sparknlp.annotator import *

document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer()\
    .setInputCols(["document"])\
    .setOutputCol("token")

xlm_roberta_loaded = XlmRoBertaEmbeddings.load(f"{MODEL_NAME}_spark_nlp")\
    .setInputCols(["document",'token'])\
    .setOutputCol("xlm_roberta")\

pipeline = Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        xlm_roberta_loaded
  ])

data = spark.createDataFrame([['William Henry Gates III (born October 28, 1955) is an American business magnate, software developer, investor,and philanthropist.']]).toDF("text")
model = pipeline.fit(data)
result = model.transform(data)

In [12]:
result.selectExpr("explode(xlm_roberta.embeddings) as embeddings").show()

+--------------------+
|          embeddings|
+--------------------+
|[0.017810468, 0.1...|
|[-0.005121568, 0....|
|[0.0051700654, 0....|
|[0.006572605, 0.1...|
|[-0.028698405, 0....|
|[-0.0055642026, 0...|
|[-0.017623411, 0....|
|[-0.11884114, 0.0...|
|[-0.08074665, 0.1...|
|[-0.034696482, 0....|
|[-0.06809629, 0.1...|
|[-0.050851095, 0....|
|[-0.006526501, 0....|
|[-0.02970995, 0.1...|
|[0.011362048, 0.2...|
|[0.044628035, 0.5...|
|[0.022998871, 0.2...|
|[0.017431622, 0.2...|
|[-0.02495086, 0.1...|
|[-0.03151579, 0.1...|
+--------------------+
only showing top 20 rows



That's it! You can now go wild and use hundreds of XLM-RoBERTa models from HuggingFace 🤗 in Spark NLP 🚀
